<a href="https://colab.research.google.com/github/LittleHaku/spotify-reviews-sentiment-analysis/blob/main/Spotify_User_Reviews_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

This is my first project regarding NLP. The goal with this project is to learn how to